In [6]:
import os, sys
import random
import json
import nltk 
import csv
import nltk  # $ pip install nltk
from nltk.stem import PorterStemmer
from nltk.corpus import cmudict  # >>> nltk.download('cmudict')
from nltk.tokenize import word_tokenize
from spellchecker import SpellChecker

from beliefbank_data.utils import generate_assertion, generate_question, find_constraints

In [2]:
constraints_path = "beliefbank_data/constraints_v2.json"
facts_path = "beliefbank_data/silver_facts.json"

In [3]:
constraints = json.load(open(constraints_path))
facts = json.load(open(facts_path))

In [4]:
statements = [(entity, relation, label == 'yes')
              for entity, relations in facts.items() 
              for relation, label in relations.items()]
statements[:2]

[('american bison', 'IsA,mammal', True),
 ('american bison', 'IsA,american bison', True)]

In [7]:
for base in statements:
    entity, relation, true = base
    
    filter_dict = {
        'source': relation,
        'direction': 'forward',
    }
    selected_constraints = find_constraints(constraints, filter_dict=filter_dict)
    c = random.choice(selected_constraints)
    contra = (entity, c['target'], not c['weight'] == 'yes_yes')
    print(base, contra)
    break

('american bison', 'IsA,mammal', True) ('american bison', 'IsA,sugar', True)


In [15]:
questions, answers = zip(*[generate_question(*base), generate_question(*contra)])
questions = list(questions)
answers = list(answers)

In [16]:
print(questions)
print(answers)

['Is a american bison a mammal?', 'Is a american bison a sugar?']
['Yes', 'Yes']


In [18]:
# run QA model on questions to get answers and confidences

In [ ]:
# run NLI model on assertions from QA output